In [8]:
import numpy as np
from numpy import array, zeros, diag, ones, sin, cos, tan, linspace, dot, pi
from numpy.random import uniform
from mytraj import MyTraj 
import time
from scipy import integrate
from numpy import isnan, pi, isinf
import pandas as pd
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score

# In order to use modified modules without restarting
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def predict(X, model, device, method="MAX"):
    inputs = torch.from_numpy(X).to(device)
    outputs = model(inputs.float())
    
    if method=="MAX":
        _, preds = torch.max(outputs, 1)
        return preds
    else:
        prob = nn.Softmax(dim=1)
        return prob(outputs)
    
    
    
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop_layer = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(37, 400)
        self.fc2 = nn.Linear(400, 200)
        self.fc3 = nn.Linear(200, 100)
        self.fc4 = nn.Linear(100, 50)
        self.fc5 = nn.Linear(50, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc3(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc4(x))
        x = self.drop_layer(x)
        x = self.fc5(x)
        return x
    
    
def model_parameters():
    g = 9.81
    m = 1.52
    Ixx, Iyy, Izz = 0.0347563, 0.0458929, 0.0977
    I1 = (Iyy - Izz) / Ixx
    I2 = (Izz - Ixx) / Iyy
    I3 = (Ixx - Iyy) / Izz
    Jr = 0.0001
    l = 0.09
    b = 8.54858e-6
    d = 1.6e-2
    
    return g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d

def model_dynamics(t, state, U):
    g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d = model_parameters()
    #states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]
    x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot = state
    U1, U2, U3, U4 = U
    
    if (d*U1 - 2*d*U3 - b*U4) < 0:
        omega1 = - np.sqrt(- d*U1 + 2*d*U3 + b*U4) / (2*np.sqrt(b*d))
    else:
        omega1 = - np.sqrt(d*U1 - 2*d*U3 - b*U4) / (2*np.sqrt(b*d))
        
    if (d*U1 - 2*d*U2 + b*U4) < 0:
        omega2 = -np.sqrt(-d*U1 + 2*d*U2 - b*U4) / (2*np.sqrt(b*d))
    else:
        omega2 = -np.sqrt(d*U1 - 2*d*U2 + b*U4) / (2*np.sqrt(b*d))
    
    
    if (d*U1 + 2*d*U3 - b*U4) < 0:
        omega3 = -np.sqrt(-d*U1 - 2*d*U3 + b*U4) / (2*np.sqrt(b*d))
    else:
        omega3 = -np.sqrt(d*U1 + 2*d*U3 - b*U4) / (2*np.sqrt(b*d))
        
    if (d*U1 + 2*d*U2 + b*U4) < 0:
        omega4 = -np.sqrt(-d*U1 - 2*d*U2 - b*U4) / (2*np.sqrt(b*d))
    else:
        omega4 = -np.sqrt(d*U1 + 2*d*U2 + b*U4) / (2*np.sqrt(b*d))
        
    
    omega = (omega2 + omega4 - omega1 - omega3)
    
    state_dot = np.zeros(12)
    state_dot[0] = x_dot
    state_dot[1] = y_dot
    state_dot[2] = z_dot
    state_dot[3] = phi_dot
    state_dot[4] = theta_dot
    state_dot[5] = psi_dot
    state_dot[6] = (cos(phi)*sin(theta)*cos(psi) + sin(phi)*sin(psi))*U[0]/m
    state_dot[7] = (cos(phi)*sin(theta)*sin(psi) - sin(phi)*cos(psi))*U[0]/m
    state_dot[8] = -g + cos(phi)*cos(theta)*U[0]/m
    state_dot[9] = theta_dot*psi_dot*I1 - Jr / Ixx * theta_dot * omega  + l/Ixx*U[1]
    state_dot[10] = phi_dot*psi_dot*I2 + Jr / Iyy * phi_dot * omega + l/Iyy*U[2]
    state_dot[11] = phi_dot*theta_dot*I3 + 1/Izz*U[3]
    
    return state_dot
    
def backstepping(A1, A2, A3, A4, A5, A6, U_list, ref_traj, states):
    g, m, Ixx, Iyy, Izz, I1, I2, I3, Jr, l, b, d = model_parameters()
    
    U1, U2, U3, U4 = U_list
    
    #states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]
    x, y, z = states[0], states[1], states[2]
    phi, theta, psi = states[3], states[4], states[5]
    x_dot, y_dot, z_dot = states[6], states[7], states[8]
    phi_dot, theta_dot, psi_dot = states[9], states[10], states[11]
    
#     ref_traj = [xd[i], yd[i], zd[i], xd_dot[i], yd_dot[i], zd_dot[i], 
#                 xd_ddot[i], yd_ddot[i], zd_ddot[i], xd_dddot[i], yd_dddot[i],
#                 xd_ddddot[i], yd_ddddot[i], psid[i], psid_dot[i], psid_ddot[i]]

    
    xd, yd, zd = ref_traj[0], ref_traj[1], ref_traj[2], 
    xd_dot, yd_dot, zd_dot = ref_traj[3], ref_traj[4], ref_traj[5]
    xd_ddot, yd_ddot, zd_ddot = ref_traj[6], ref_traj[7], ref_traj[8]
    xd_dddot, yd_dddot = ref_traj[9], ref_traj[10]
    xd_ddddot, yd_ddddot = ref_traj[11], ref_traj[12]
    psid, psid_dot, psid_ddot = ref_traj[13], ref_traj[14], ref_traj[15]
    
    x1, x2, x3 = array([[x], [y]]), array([[x_dot], [y_dot]]), array([[phi], [theta]])
    x4, x5, x6 = array([[phi_dot], [theta_dot]]), array([[psi], [z]]), array([[psi_dot], [z_dot]])
    
    g0 = array([[np.cos(psi), np.sin(psi)],  [np.sin(psi), -np.cos(psi)]])
    g0_inv = array([[np.cos(psi), np.sin(psi)],  [np.sin(psi), -np.cos(psi)]])
    
    g1 = array([[theta_dot*psi_dot*I1],  [phi_dot*psi_dot*I2]])
    g2 = array([[phi_dot*theta_dot*I3],  [-g]])
    
    l0 = array([[np.cos(phi)*np.sin(theta)],  [np.sin(phi)]])*U1/m 
    dl0_dx3 = array([[-np.sin(phi)*np.sin(theta), np.cos(phi)*np.cos(theta)],  [np.cos(phi), 0]])*U1/m 
    dl0_dx3_inv = array([[0, 1/np.cos(phi)],  [1/np.cos(theta)*1/np.cos(phi), 1/np.cos(phi)*np.tan(theta)*np.tan(phi)]])*m/U1 
    dl0_dx3_inv_dot = array([[0, 1/np.cos(phi)*np.tan(phi)*phi_dot], 
                             [1/np.cos(theta)*1/np.cos(phi)*(np.tan(theta)*theta_dot + np.tan(phi)*phi_dot), 1/np.cos(phi)*((1/np.cos(theta))**2*np.tan(phi)*theta_dot + (-1+2*(1/np.cos(phi))**2)*np.tan(theta)*phi_dot)]])*m/U1 
                
#     Omega_square = Omega_coef_inv * abs([U1/b  U2/b  U3/b  U4/d]) 
#     Omega_param = sqrt(Omega_square) 
#     omega = Omega_param(2) + Omega_param[3] - Omega_param(1) - Omega_param(3) 

#     h1 = [-Jr/Ixx*theta_dot*omega  Jr/Iyy*phi_dot*omega] 
    h1 = 0 
    k1 = diag([l/Ixx, l/Iyy]) 
    k1_inv = diag([Ixx/l, Iyy/l]) 
    k2 = diag([1/Izz, np.cos(phi)*np.cos(theta)/m]) 
    k2_inv = diag([Izz, m/(np.cos(phi)*np.cos(theta))]) 
    
    x1d = array([[xd], [yd]])  
    x1d_dot = array([[xd_dot], [yd_dot]]) 
    x1d_ddot = array([[xd_ddot], [yd_ddot]]) 
    x1d_dddot = array([[xd_dddot], [yd_dddot]]) 
    x1d_ddddot = array([[xd_ddddot], [yd_ddddot]]) 
    
    x5d = array([[psid], [zd]])
    x5d_dot = array([[psid_dot], [zd_dot]]) 
    x5d_ddot = array([[psid_ddot], [zd_ddot]]) 
    
    z1 = x1d - x1 
    v1 = x1d_dot + dot(A1,z1) 
    z2 = v1 - x2 
    z1_dot = -dot(A1,z1) + z2 
    v1_dot = x1d_ddot + dot(A1,z1_dot) 
    v2 = dot(g0_inv, z1 + v1_dot + dot(A2,z2)) 
    z3 = v2 - l0  
    z2_dot = -z1 - dot(A2,z2) + dot(g0,z3) 
    z1_ddot = -dot(A1,z1_dot) + z2_dot 
    v1_ddot = x1d_dddot + dot(A1, z1_ddot) 
    v2_dot = dot(g0_inv, z1_dot + v1_ddot + dot(A2,z2_dot)) 
    v3 = dot(dl0_dx3_inv, dot(g0.T,z2) + v2_dot + dot(A3, z3)) 
    z4 = v3 - x4 
    z3_dot = -dot(g0.T, z2) - dot(A3,z3) + dot(dl0_dx3, z4) 
    z2_ddot = - z1_dot - dot(A2, z2_dot) + dot(g0, z3_dot) 
    z1_dddot = -dot(A1, z1_ddot) + z2_ddot 
    v1_dddot = x1d_ddddot + dot(A1, z1_dddot) 
    v2_ddot = dot(g0_inv, z1_ddot + v1_dddot + dot(A2, z2_ddot)) 
    v3_dot = dot(dl0_dx3_inv, dot(g0.T, z2_dot) + v2_ddot + dot(A3, z3_dot)) + dot(dl0_dx3_inv_dot, dot(g0.T, z2) + v2_dot + dot(A3, z3))
    l1 = dot(k1_inv, dot(dl0_dx3.T, z3) + v3_dot - g1 - h1 + dot(A4, z4)).ravel()
    
    z5 = x5d - x5 
    v5 = x5d_dot + dot(A5, z5) 
    z6 = v5 - x6 
    z5_dot = - dot(A5, z5) + z6 
    v5_dot = x5d_ddot + dot(A5, z5_dot) 
    l2 = dot(k2_inv, z5 + v5_dot - g2 + dot(A6, z6)).ravel()
    
    U1, U2, U3, U4 = l2[1], l1[0], l1[1], l2[0]
    
    U1 = np.clip(U1, 1.0, 1e2)
    U2 = np.clip(U2, -1e3, 1e3)
    U3 = np.clip(U3, -1e3, 1e3)
    U4 = np.clip(U4, -1e2, 1e2)
    
    U = np.array([U1, U2, U3, U4])
    
    return U

In [3]:
def write_stats(stats): #testno,stats_columns
    df_stats = pd.DataFrame([stats], columns=stats_columns)
    df_stats.to_csv(stats_filename, mode='a', index=False,header=not os.path.isfile(stats_filename))
    
    
def get_control_input(cont, Controllers, U0, current_traj, state):
    if (cont == Controllers[0]): #Backstepping_1
        A1, A2, A3 = 15*diag([1,1]), 10*diag([1,1]), 15*diag([1,1]) 
        A4, A5, A6 = 10*diag([1,1]), 15*diag([1,1]), 10*diag([1,1]) 
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state) 
    elif (cont == Controllers[1]): #Backstepping_2
        A1, A2, A3 = 10*diag([1,1]), 5*diag([1,1]), 10*diag([1,1]) 
        A4, A5, A6 = 5*diag([1,1]), 10*diag([1,1]), 5*diag([1,1])
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state) 
    elif (cont == Controllers[2]): #Backstepping_3
        A1, A2, A3 = 5*diag([1,1]), 3*diag([1,1]), 10*diag([1,1]) 
        A4, A5, A6 = 7*diag([1,1]), 1*diag([1,1]), 1*diag([1,1])  
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state)
    elif (cont == Controllers[3]): #Backstepping_4
        A1, A2, A3 = 2*diag([1,1]), 5*diag([1,1]), 2*diag([1,1]) 
        A4, A5, A6 = 5*diag([1,1]), 2*diag([1,1]), 5*diag([1,1]) 
        U = backstepping(A1, A2, A3, A4, A5, A6, U0, current_traj, state)
        
    return U
    
def quad_sim(Tf, state0, ref_traj, scaler=None, model=None, device=None, status="TEST", method="MAX"):
#     states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]


    N = ref_traj.shape[0]
    costValue = 0
    U = [1,0,0,0]

    Controllers = ["Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
    single_shot = ["Back"] # just a check flight to test the coefficients
    count_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
    U_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
    state = np.copy(state0)
    
    for i in range(N):
        
        #     ref_traj = [xd[i], yd[i], zd[i], xd_dot[i], yd_dot[i], zd_dot[i], 
        #                 xd_ddot[i], yd_ddot[i], zd_ddot[i], xd_dddot[i], yd_dddot[i],
        #                 xd_ddddot[i], yd_ddddot[i], psid[i], psid_dot[i], psid_ddot[i]]
        prev_traj_index = np.maximum(0, i-1)
        
        current_traj = [ref_traj[i][0], ref_traj[i][1], ref_traj[i][2], ref_traj[i][3], ref_traj[i][4], ref_traj[i][5], 
                        ref_traj[i][6], ref_traj[i][7], ref_traj[i][8], ref_traj[i][9], ref_traj[i][10], ref_traj[i][11],
                        ref_traj[i][12], ref_traj[i][13], ref_traj[i][14], ref_traj[i][15]]
        
        prev_traj = [ref_traj[prev_traj_index][0], ref_traj[prev_traj_index][1], ref_traj[prev_traj_index][2], ref_traj[prev_traj_index][3], ref_traj[prev_traj_index][4], ref_traj[prev_traj_index][5], 
                     ref_traj[prev_traj_index][6], ref_traj[prev_traj_index][7], ref_traj[prev_traj_index][8], ref_traj[prev_traj_index][9], ref_traj[prev_traj_index][10], ref_traj[prev_traj_index][11],
                     ref_traj[prev_traj_index][12], ref_traj[prev_traj_index][13], ref_traj[prev_traj_index][14], ref_traj[prev_traj_index][15]]

        U0 = np.copy(U)
        
        #Test part
        if status == "TEST":
             
#             X_test = np.array([state[0]-current_traj[0], state[1]-current_traj[1], state[2]-current_traj[2], 
#                                state[6], state[7], state[8], acc0[0], acc0[1], acc0[2], 
#                                state[3], state[4], state[5], state[9], state[10], state[11], 
#                                state[0], state[1], state[2], state[6], state[7], state[8],
#                                state[3], state[4], state[5], state[9], state[10], state[11], Tf]).reshape(1,-1)
            X_test = scaler.transform(X_test)
        
            if METHOD == "MAX":
                pred = predict(X_test, model, device, method)
                cont = Controller[pred[0]]
                count_dict[cont] += 1
                
            elif METHOD == "MIX":
                pass
                
            elif METHOD == "DICE":
                pass
            
        elif status == "DATA_COLLECTION":
            cost_dict = {"Backstepping_1": 0, "Backstepping_2": 0, "Backstepping_3": 0, "Backstepping_4": 0}
            state_dict = {"Backstepping_1": [], "Backstepping_2": [], "Backstepping_3": [], "Backstepping_4": []}
            for cont in Controllers:  
                U = get_control_input(cont, Controllers, U0, current_traj, state)
                sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, 1e-3), y0=state, args = (U,))
                new_state = sol.y[:,-1]
                state_dict[cont] = new_state
                U_dict[cont] = U
                
                if (np.abs(new_state[3]) > pi/2)  | (np.abs(new_state[4]) > pi/2):
                    costValue = 1e12
                    cost_dict[cont] = costValue
                    continue

            #     states: [x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot]

                position_tracking_error = np.power((current_traj[0]-new_state[0]),2) + np.power((current_traj[1]-new_state[1]),2) + np.power((current_traj[2]-new_state[2]),2)
                velocity_tracking_error = np.power((current_traj[3]-new_state[6]),2) + np.power((current_traj[4]-new_state[7]),2) + np.power((current_traj[5]-new_state[8]),2)
                angular_error = new_state[3]**2 + new_state[4]**2 + new_state[5]**2
                cont_input = U[0]**2 + U[1]**2 + U[2]**2 + U[3]**2
                costValue = (position_tracking_error +  .25*angular_error + 0.*5e-5*cont_input)
                cost_dict[cont] = costValue
                
                
            min_cost_index = min(cost_dict.items(), key=lambda x: x[1])[0]
            count_dict[min_cost_index] += 1   
            U = U_dict[min_cost_index]
            
            t_current = Tf / N * i
            time_rate = float(t_current / Tf)
            Upr_abs_sum = np.sum(np.abs(U0))
            

            write_stats([state[0], state[1], state[2], state[6], state[7], state[8], state[3], state[4], state[5], state[9], state[10], state[11],
                        current_traj[0], current_traj[1], current_traj[2], current_traj[3], current_traj[4], current_traj[5], current_traj[6], current_traj[7], current_traj[8],
                        state[0]-current_traj[0], state[1]-current_traj[1], state[2]-current_traj[2], time_rate, t_current, Tf, 
                        prev_traj[0], prev_traj[1], prev_traj[2], prev_traj[3], prev_traj[4], prev_traj[5], prev_traj[6], prev_traj[7], prev_traj[8], Upr_abs_sum,
                        min_cost_index])
            
            state = state_dict[min_cost_index]


    print ("")
    print ("How many times Backstepping_1 is called?: ", count_dict["Backstepping_1"])
    print ("How many times Backstepping_2 is called?: ", count_dict["Backstepping_2"])
    print ("How many times Backstepping_3 is called?: ", count_dict["Backstepping_3"])
    print ("How many times Backstepping_4 is called?: ", count_dict["Backstepping_4"])
    print ("Final state, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3}, vx: {6:.3}, vy: {7:.3}, vz: {8:.3}, p: {9:.3}, q: {10:.3}, r: {11:.3}".format(state[0],state[1],state[2],state[3],state[4],state[5], state[6],state[7],state[8], state[9],state[10],state[11]))

In [7]:
stats_columns = ['x0', 'y0', 'z0', 'x_dot0','y_dot0','z_dot0', 'phi0','theta0','yaw0', 'phi_dot0','theta_dot0','yaw_dot0', 
                 'xf', 'yf', 'zf', 'x_dotf','y_dotf','z_dotf','x_ddotf','y_ddotf','z_ddotf',
                 'pos_diffx','pos_diffy','pos_diffz','time_rate','t', 'Tf', 
                 'xp', 'yp', 'zp', 'x_dotp','y_dotp','z_dotp','x_ddotp','y_ddotp','z_ddotp', 'u_abs_p',
                 'controller_ID']
stats_filename = "flight.csv"

STATUS = "DATA_COLLECTION"
# STATUS = "SINGLE_SHOT"
# STATUS = "TEST"
METHOD = "MAX"
# METHOD = "MIX"
# METHOD = "DICE"

dtau = 1e-3

if STATUS == "TEST":        
    with open('dataset.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
        X_train, X_val, X_test, y_train, y_val, y_test = pickle.load(f)

    #To normalize data
    scaler = StandardScaler()
    scaler.fit(X_train)

    #Neural network
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = Net()
    model = model.to(device)
    model.load_state_dict(torch.load('best_model_0.86.pt'))
    model.eval()


K = 50 # how many different trajectories to be followed

v_lim = 5.0
acc_lim = 2.5
ang_lim = 75*np.pi/180
ang_vel_lim = 1.0
pos_lim = 15

U_list = []

for j in range(K):
    pos0 = [uniform(low=-2.0, high=2.0), uniform(low=-2.0, high=2.0), uniform(low=0.0, high=10.0)]
    vel0 = [uniform(low=-v_lim, high=v_lim), uniform(low=-v_lim, high=v_lim), uniform(low=-v_lim, high=v_lim)]
    acc0 = [uniform(low=-acc_lim, high=acc_lim), uniform(low=-acc_lim, high=acc_lim), uniform(low=-acc_lim, high=acc_lim)]
    ang0 = [uniform(low=-ang_lim, high=ang_lim), uniform(low=-ang_lim, high=ang_lim), uniform(low=-1.57, high=1.57)]
    ang_vel0 = [uniform(low=-ang_vel_lim, high=ang_vel_lim), uniform(low=-ang_vel_lim, high=ang_vel_lim), uniform(low=-ang_vel_lim, high=ang_vel_lim)]
    posf = [uniform(low=-pos_lim, high=pos_lim), uniform(low=-pos_lim, high=pos_lim), uniform(low=0.0, high=1.5*pos_lim)]
    velf = [0.,0.,0.]#[uniform(low=0, high=0.5), uniform(low=0, high=0.5), uniform(low=0, high=0.5)]
    accf = [0.,0.,0.]#[uniform(low=0, high=0.15), uniform(low=0, high=0.15), uniform(low=0, high=0.15)]
    Tf = uniform(low=5, high=15)
    
    traj = MyTraj(gravity = -9.81)
    trajectory = traj.givemetraj(pos0, vel0, acc0, posf, velf, accf, Tf)
    
    N = int(Tf/dtau)
    t = linspace(0,Tf,num = N)
    
    xd, yd, zd, psid = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_dot, yd_dot, zd_dot, psid_dot = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_ddot, yd_ddot, zd_ddot, psid_ddot = zeros(t.shape), zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_dddot, yd_dddot, zd_dddot = zeros(t.shape), zeros(t.shape), zeros(t.shape)
    xd_ddddot, yd_ddddot, zd_ddddot = zeros(t.shape), zeros(t.shape), zeros(t.shape)
    
    i = 0
    ts = 0
    
    for i in range(N):
        pos_des, vel_des, acc_des = traj.givemepoint(trajectory, ts)
        
        xd[i], yd[i], zd[i] = pos_des[0], pos_des[1], pos_des[2]
        xd_dot[i], yd_dot[i], zd_dot[i] = vel_des[0], vel_des[1], vel_des[2]
        xd_ddot[i], yd_ddot[i], zd_ddot[i] = acc_des[0], acc_des[1], acc_des[2]
        ts += dtau
    
    
    ref_traj = np.c_[xd, yd, zd, xd_dot, yd_dot, zd_dot, xd_ddot, yd_ddot, zd_ddot,
                     xd_dddot, yd_dddot, xd_ddddot, yd_ddddot,
                     psid, psid_dot, psid_ddot]
    
    state0 = np.array([pos0[0], pos0[1], pos0[2], ang0[0], ang0[1], ang0[2], 
                       vel0[0], vel0[1], vel0[2], ang_vel0[0], ang_vel0[1], ang_vel0[2]])
    
    print ("")
    print ("-"*25)
    print ("Init, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3}, vx: {6:.3}, vy: {7:.3}, vz: {8:.3}, p: {9:.3}, q: {10:.3}, r: {11:.3}".format(pos0[0], pos0[1], pos0[2], ang0[0], ang0[1], ang0[2], vel0[0], vel0[1], vel0[2], ang_vel0[0], ang_vel0[1], ang_vel0[2]))
    print ("Goal, x: {0:.3}, y: {1:.3}, z: {2:.3}, vx: {3:.3}, vy: {4:.3}, vz: {5:.3} in {6:.3} s.".format(posf[0], posf[1], posf[2], velf[0], velf[1], velf[2], Tf))
    
    if STATUS == "DATA_COLLECTION" or STATUS == "SINGLE_SHOT":
        quad_sim(Tf, state0, ref_traj, status=STATUS)
    elif STATUS == "TEST":
        quad_sim(Tf, state0, ref_traj, scaler=scaler, model=model, device=device, status=STATUS, method=METHOD)


-------------------------
Init, x: -0.11, y: -0.763, z: 9.58, phi: -0.131, theta: -0.912, psi: 1.03, vx: -2.32, vy: -2.74, vz: -1.4, p: 0.293, q: 0.244, r: 0.398
Goal, x: -0.991, y: 4.05, z: 21.9, vx: 0.0, vy: 0.0, vz: 0.0 in 12.0 s.

How many times Backstepping_1 is called?:  5782
How many times Backstepping_2 is called?:  282
How many times Backstepping_3 is called?:  595
How many times Backstepping_4 is called?:  5390
Final state, x: -0.991, y: 4.05, z: 21.9, phi: 3.25e-05, theta: 8.42e-06, psi: -9.39e-10, vx: 0.000197, vy: -0.000755, vz: -5.57e-07, p: -0.106, q: -0.0208, r: -7.91e-10

-------------------------
Init, x: -0.703, y: -0.464, z: 2.3, phi: -0.00327, theta: 0.167, psi: 0.434, vx: -0.371, vy: 4.34, vz: -2.68, p: -0.46, q: -0.373, r: -0.997
Goal, x: -0.585, y: -6.35, z: 18.1, vx: 0.0, vy: 0.0, vz: 0.0 in 5.52 s.

How many times Backstepping_1 is called?:  2031
How many times Backstepping_2 is called?:  75
How many times Backstepping_3 is called?:  454
How many times Backst


-------------------------
Init, x: 1.38, y: 1.64, z: 8.92, phi: -0.545, theta: -0.93, psi: 0.789, vx: 0.416, vy: 4.16, vz: -3.06, p: 0.521, q: 0.976, r: -0.0554
Goal, x: -3.66, y: -8.66, z: 7.28, vx: 0.0, vy: 0.0, vz: 0.0 in 5.5 s.

How many times Backstepping_1 is called?:  2063
How many times Backstepping_2 is called?:  72
How many times Backstepping_3 is called?:  654
How many times Backstepping_4 is called?:  2707
Final state, x: -3.66, y: -8.65, z: 7.28, phi: -0.000304, theta: 5.85e-05, psi: 1.63e-07, vx: 0.00398, vy: 0.0125, vz: -1.48e-05, p: 0.813, q: -0.272, r: 1.87e-07

-------------------------
Init, x: -0.19, y: -1.3, z: 0.908, phi: 0.763, theta: -1.2, psi: 1.3, vx: -2.86, vy: -2.18, vz: 3.03, p: 0.0621, q: 0.446, r: -0.831
Goal, x: -5.08, y: -8.2, z: 3.73, vx: 0.0, vy: 0.0, vz: 0.0 in 13.9 s.

How many times Backstepping_1 is called?:  6326
How many times Backstepping_2 is called?:  473
How many times Backstepping_3 is called?:  1178
How many times Backstepping_4 is called


-------------------------
Init, x: 1.57, y: -0.103, z: 9.9, phi: 0.612, theta: -0.39, psi: 0.688, vx: 1.56, vy: 4.16, vz: 2.76, p: -0.677, q: -0.227, r: 0.629
Goal, x: -6.84, y: 4.25, z: 22.1, vx: 0.0, vy: 0.0, vz: 0.0 in 6.11 s.

How many times Backstepping_1 is called?:  2220
How many times Backstepping_2 is called?:  115
How many times Backstepping_3 is called?:  670
How many times Backstepping_4 is called?:  3105
Final state, x: -6.83, y: 4.25, z: 22.1, phi: -1.07e-05, theta: 0.000172, psi: 8.15e-08, vx: 0.00574, vy: 0.00386, vz: -2.41e-06, p: 0.244, q: -0.437, r: 1.09e-07

-------------------------
Init, x: -0.49, y: 1.69, z: 8.72, phi: 0.469, theta: 0.531, psi: 0.433, vx: -4.78, vy: 1.58, vz: -0.993, p: 0.28, q: -0.745, r: 0.716
Goal, x: 5.93, y: -3.51, z: 5.57, vx: 0.0, vy: 0.0, vz: 0.0 in 9.0 s.

How many times Backstepping_1 is called?:  3781
How many times Backstepping_2 is called?:  149
How many times Backstepping_3 is called?:  645
How many times Backstepping_4 is called?:


-------------------------
Init, x: 0.55, y: 1.14, z: 8.82, phi: -0.541, theta: -0.565, psi: 1.46, vx: -3.97, vy: 3.17, vz: -3.98, p: 0.314, q: 0.275, r: -0.971
Goal, x: 5.01, y: -1.93, z: 13.0, vx: 0.0, vy: 0.0, vz: 0.0 in 6.09 s.

How many times Backstepping_1 is called?:  2922
How many times Backstepping_2 is called?:  71
How many times Backstepping_3 is called?:  835
How many times Backstepping_4 is called?:  2264
Final state, x: 5.0, y: -1.93, z: 13.0, phi: 0.000708, theta: 0.000682, psi: -3.35e-08, vx: -0.00256, vy: 0.00287, vz: -2.84e-05, p: 0.274, q: 0.281, r: 7.36e-07

-------------------------
Init, x: 0.984, y: 0.895, z: 9.25, phi: -1.31, theta: -0.774, psi: -0.259, vx: 2.02, vy: -0.586, vz: -3.78, p: 0.716, q: -0.167, r: -0.0566
Goal, x: 2.68, y: -6.23, z: 0.358, vx: 0.0, vy: 0.0, vz: 0.0 in 11.1 s.

How many times Backstepping_1 is called?:  5234
How many times Backstepping_2 is called?:  195
How many times Backstepping_3 is called?:  746
How many times Backstepping_4 is c

In [ ]:
# #Deneme
# Controllers = ["Backstepping_1", "Backstepping_2", "Backstepping_3", "Backstepping_4"]
# U0 = [1,0,0,0]
# state = state0
# for i in range(100):
#     current_traj = [ref_traj[i][0], ref_traj[i][1], ref_traj[i][2], ref_traj[i][3], ref_traj[i][4], ref_traj[i][5], 
#                             ref_traj[i][6], ref_traj[i][7], ref_traj[i][8], ref_traj[i][9], ref_traj[i][10], ref_traj[i][11],
#                             ref_traj[i][12], ref_traj[i][13], ref_traj[i][14], ref_traj[i][15]]
#     U = get_control_input('Backstepping_4', Controllers, U0, current_traj, state)
#     sol = integrate.solve_ivp(fun=model_dynamics, t_span=(0, 1e-3), y0=state, args = (U,))
#     print ("U: ", U)
#     print ("target: ", current_traj[0:3])
#     print ("state: ", sol.y[:,-1])
#     U0 = U
#     state = sol.y[:, -1]

In [ ]:
T = 5.                           #simulation time 
dtau = 1e-3                      #sampling time for controller
dt = 1e-4                        #sampling time for solver 
N = int(T / dtau)                #how many times controller will be designed
Nsolver = int(dtau / dt)         #how many times solver is executed in each step
state_list = []
t = linspace(0,T,num = N)


# #Trajectory tracking
# freq, amp = 1, 5
# xd = amp*sin(freq*t)
# # yd = amp*cos(freq*t)
# yd = amp*sin(freq*t)
# # zd = amp*t
# zd = amp*sin(freq*t)
# psid = zeros(xd.shape)
# xd_dot = amp*freq*cos(freq*t)
# # yd_dot = -amp*freq*sin(freq*t)
# yd_dot = amp*freq*cos(freq*t)
# # zd_dot = amp*ones(xd.shape)
# zd_dot = amp*freq*cos(freq*t)
# psid_dot = zeros(xd.shape)
# xd_ddot = -amp*freq**2*sin(freq*t)
# # yd_ddot = -amp*freq**2*cos(freq*t)
# yd_ddot = -amp*freq**2*sin(freq*t)
# # zd_ddot = zeros(xd.shape)
# zd_ddot = -amp*freq**2*sin(freq*t)
# psid_ddot = zeros(xd.shape)
# xd_dddot = -amp*freq**3*cos(freq*t)
# # yd_dddot = amp*freq**3*sin(freq*t)
# yd_dddot = -amp*freq**3*cos(freq*t)
# # zd_dddot = zeros(xd.shape)
# zd_dddot = -amp*freq**3*cos(freq*t)
# xd_ddddot = amp*freq**4*sin(freq*t)
# # yd_ddddot = amp*freq**4*cos(freq*t)
# yd_ddddot = amp*freq**4*sin(freq*t)
# # zd_ddddot = zeros(xd.shape)
# zd_ddddot = amp*freq**4*sin(freq*t)


# Const
xd = 5*ones(t.shape)
yd = 7*ones(t.shape)
zd = 4*ones(t.shape)
psid = zeros(t.shape)
xd_dot = zeros(t.shape)
yd_dot = zeros(t.shape)
zd_dot = zeros(t.shape)
psid_dot = zeros(t.shape)
xd_ddot = zeros(t.shape)
yd_ddot = zeros(t.shape)
zd_ddot = zeros(t.shape)
psid_ddot = zeros(t.shape)
xd_dddot = zeros(t.shape)
yd_dddot = zeros(t.shape)
zd_dddot = zeros(t.shape)
xd_ddddot = zeros(t.shape)
yd_ddddot = zeros(t.shape)
zd_ddddot = zeros(t.shape)


A1, A2, A3 = 5*diag([1,1]), 3*diag([1,1]), 10*diag([1,1]) 
A4, A5, A6 = 7*diag([1,1]), 1*diag([1,1]), 1*diag([1,1]) 

state0 = array([0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.,0.])    #initial states
state_list.append(state0)

U1, U2, U3, U4 = 1, 0, 0, 0
U = np.array([U1, U2, U3, U4])
state = np.copy(state0)

for i in range(1,N):    
    ref_traj = np.array([xd[i], yd[i], zd[i], xd_dot[i], yd_dot[i], zd_dot[i], 
                xd_ddot[i], yd_ddot[i], zd_ddot[i], xd_dddot[i], yd_dddot[i],
                xd_ddddot[i], yd_ddddot[i], psid[i], psid_dot[i], psid_ddot[i]]).ravel()
    
    U = backstepping(A1, A2, A3, A4, A5, A6, U, ref_traj, state) 
    state = integrate.odeint(model_dynamics, state, [0, dt], args = (U,))[1]
        
    
#     print ("Step: ", i)
#     print ("STATES: ", state)
#     print ("\n\n")
    state_list.append(state)
    
print ("-"*25)
print ("Init, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3}".format(state0[0], state0[1], state0[2], state0[3], state0[4], state0[5]))
print ("Goal, x: {0:.3}, y: {1:.3}, z: {2:.3}, phi: {3:.3}, theta: {4:.3}, psi: {5:.3} in {6:.3} s.".format(state[0], state[1], state[2], state[3]*180/pi, state[4]*180/pi, state[5]*180/pi, T))


In [ ]:
state0

In [ ]:
x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot = state

In [ ]:
ref_traj